In [44]:
import pandas as pd
import numpy as np
from itertools import product, combinations

In [2]:
# read user ratings
allRatings = pd.read_csv("ml-latest-small/ratings.csv")
allRatings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [14]:
len(allRatings.movieId.unique())

9724

In [3]:
allMovies = pd.read_csv("ml-latest-small/movies.csv")
allMovies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [4]:
# merging the rating data and movie name data based on movieId to create the master dataframe
allMovieData = pd.merge(left=allRatings, right=allMovies, on=['movieId'])

# removing all movies that has rating < 3
allMovieData = allMovieData[allMovieData['rating']>=3]
allMovieData

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
5,18,1,3.5,1455209816,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...,...,...,...
100830,610,158721,3.5,1479542491,Gen-X Cops (1999),Action|Comedy|Thriller
100832,610,160527,4.5,1479544998,Sympathy for the Underdog (1971),Action|Crime|Drama
100833,610,160836,3.0,1493844794,Hazard (2005),Action|Drama|Thriller
100834,610,163937,3.5,1493848789,Blair Witch (2016),Horror|Thriller


In [53]:
%time group_movie = allMovieData.groupby('movieId')['userId'].apply(set).reset_index(name="userGroup")
group_movie['supp'] = group_movie['userGroup'].apply(lambda x: len(x))
group_movie

Wall time: 462 ms


,movieId,userGroup,supp
0,1,"{1, 514, 5, 517, 7, 522, 524, 525, 17, 18, 19,...",199
1,2,"{512, 517, 6, 8, 523, 525, 527, 18, 19, 20, 21...",89
2,3,"{1, 6, 269, 270, 19, 150, 151, 410, 414, 32, 2...",43
3,4,"{162, 84, 6, 14}",4
4,5,"{389, 6, 521, 269, 270, 402, 147, 276, 150, 31...",38
...,...,...,...
8447,193581,{184},1
8448,193583,{184},1
8449,193585,{184},1
8450,193587,{184},1


In [9]:
minSup = 40

In [54]:
%time group_movie = group_movie[group_movie['supp']>=40].set_index('movieId')
group_movie

Wall time: 3 ms


,userGroup,supp
movieId,,
1,"{1, 514, 5, 517, 7, 522, 524, 525, 17, 18, 19,...",199
2,"{512, 517, 6, 8, 523, 525, 527, 18, 19, 20, 21...",89
3,"{1, 6, 269, 270, 19, 150, 151, 410, 414, 32, 2...",43
6,"{1, 6, 521, 11, 524, 18, 532, 23, 24, 28, 32, ...",99
7,"{513, 6, 262, 521, 14, 270, 276, 150, 280, 411...",44
...,...,...
111759,"{256, 514, 525, 15, 18, 21, 534, 279, 24, 408,...",42
112852,"{514, 393, 522, 525, 401, 18, 148, 21, 534, 15...",57
116797,"{256, 515, 517, 523, 525, 18, 148, 408, 25, 41...",47


In [38]:
group_movie.index

Int64Index([     1,      2,      3,      6,      7,     10,     11,     16,
                17,     19,
            ...
             96079,  99114, 106782, 109374, 109487, 111759, 112852, 116797,
            122904, 134130],
           dtype='int64', name='movieId', length=478)

In [57]:
group_movie['supp'][3]

43

In [61]:
x = (1,2)
y = set(x)
y

{1, 2}

In [129]:
movieData = group_movie.copy()
movieData.head()

,userGroup,supp
movieId,,
1,"{1, 514, 5, 517, 7, 522, 524, 525, 17, 18, 19,...",199
2,"{512, 517, 6, 8, 523, 525, 527, 18, 19, 20, 21...",89
3,"{1, 6, 269, 270, 19, 150, 151, 410, 414, 32, 2...",43
6,"{1, 6, 521, 11, 524, 18, 532, 23, 24, 28, 32, ...",99
7,"{513, 6, 262, 521, 14, 270, 276, 150, 280, 411...",44


In [135]:
def countMovies(movieId):
    mId = list(movieId)
    mIntersection = movieData['userGroup'][mId[0]]
    for movie in mId[1:]:
        mIntersection = mIntersection.intersection(movieData['userGroup'][movie])
    return mIntersection

In [75]:
# movieData['userGroup'][7].intersection(movieData['userGroup'][3])

{6, 32, 42, 43, 51, 58, 117, 150, 169, 179, 270, 337, 414, 470, 492, 501}

In [144]:
# check = allMovieData[(allMovieData['movieId']==1) | (allMovieData['movieId'] == 2)].reset_index()
# check = check[check['userId'].apply(lambda x: len(check[check['userId']==x])==2)]
# len(check['userId'].unique())

51

In [128]:
# allMovieData[((allMovieData['movieId']==3) | (allMovieData['movieId']==7)) & (allMovieData['userId']==6)]

In [142]:
%time group_movie_df2 = pd.Series(list(combinations(list(group_movie.index), 2))).to_frame("movieId")
%time group_movie_df2['userGroup'] = group_movie_df2['movieId'].apply(lambda x: countMovies(x))
%time group_movie_df2['supp'] = group_movie_df2['userGroup'].apply(lambda x: len(x))
group_movie_df2

Wall time: 19.9 ms
Wall time: 4.07 s
Wall time: 34.4 ms


,movieId,userGroup,supp
0,"(1, 2)","{517, 135, 140, 525, 144, 18, 19, 274, 21, 534...",51
1,"(1, 3)","{1, 269, 270, 19, 151, 414, 32, 288, 544, 169,...",27
2,"(1, 6)","{1, 385, 389, 391, 524, 140, 269, 270, 18, 274...",53
3,"(1, 7)","{270, 276, 280, 411, 541, 414, 31, 32, 169, 43...",23
4,"(1, 10)","{385, 391, 522, 524, 144, 274, 21, 534, 282, 4...",58
...,...,...,...
113998,"(112852, 122904)","{525, 18, 21, 534, 154, 414, 30, 550, 298, 305...",32
113999,"(112852, 134130)","{514, 525, 18, 534, 414, 550, 551, 298, 305, 5...",26
114000,"(116797, 122904)","{523, 525, 18, 408, 414, 550, 560, 305, 62, 65...",21
114001,"(116797, 134130)","{256, 523, 525, 18, 408, 414, 550, 551, 560, 3...",24


In [143]:
group_movie_df2 = group_movie_df2[group_movie_df2['supp'] >= 40].reset_index().drop(columns=['index'])
group_movie_df2

,movieId,userGroup,supp
0,"(1, 2)","{517, 135, 140, 525, 144, 18, 19, 274, 21, 534...",51
1,"(1, 6)","{1, 385, 389, 391, 524, 140, 269, 270, 18, 274...",53
2,"(1, 10)","{385, 391, 522, 524, 144, 274, 21, 534, 282, 4...",58
3,"(1, 32)","{524, 529, 18, 19, 32, 33, 544, 555, 45, 46, 5...",92
4,"(1, 34)","{509, 514, 517, 5, 263, 391, 140, 525, 144, 27...",68
...,...,...,...
10341,"(79132, 112852)","{514, 393, 522, 525, 18, 21, 534, 154, 414, 30...",42
10342,"(89745, 91529)","{131, 393, 522, 525, 528, 18, 21, 279, 408, 41...",42
10343,"(89745, 112852)","{393, 522, 525, 18, 148, 21, 534, 154, 414, 55...",41
10344,"(91529, 109487)","{2, 514, 525, 141, 18, 21, 279, 408, 414, 30, ...",42


In [167]:
assoc_rule = list()

In [168]:
# 2 association rule
assoc_rule.extend(list(group_movie_df2['movieId']))
len(assoc_rule)

10346

In [182]:
# combine = set()
# for x in list(group_movie_df2['movieId'].apply(lambda x: list(x))):
#     combine = combine.union(set(x))
# combine = list(combine)
# len(combine)

356

In [188]:
com = set()
for x in assoc_rule:
    com = com.union(set(x))
len(com)

356

In [191]:
# 3 association
%time group_movie_df3 = pd.Series(list(combinations(list(com), 3))).to_frame("movieId")
%time group_movie_df3['userGroup'] = group_movie_df3['movieId'].apply(lambda x: countMovies(x))
%time group_movie_df3['supp'] = group_movie_df3['userGroup'].apply(lambda x: len(x))
group_movie_df3

Wall time: 1.45 s
Wall time: 7min 2s
Wall time: 2.03 s


,movieId,userGroup,supp
0,"(1, 2, 33794)","{525, 18, 274, 21, 534, 414, 561, 177, 570, 57...",23
1,"(1, 2, 1028)","{517, 140, 525, 144, 414, 160, 169, 304, 177, ...",22
2,"(1, 2, 6)","{480, 353, 202, 140, 559, 561, 18, 274, 373, 4...",16
3,"(1, 2, 3081)","{160, 480, 483, 103, 561, 274, 18, 19, 177, 24...",14
4,"(1, 2, 10)","{144, 274, 21, 534, 411, 414, 559, 304, 186, 5...",25
...,...,...,...
7456415,"(7147, 7153, 35836)","{288, 318, 480, 226, 483, 610, 232, 105, 141, ...",15
7456416,"(2028, 3052, 7153)","{514, 522, 141, 18, 274, 28, 414, 166, 167, 42...",32
7456417,"(2028, 3052, 35836)","{318, 483, 68, 356, 166, 105, 489, 141, 239, 4...",14
7456418,"(2028, 7153, 35836)","{129, 141, 274, 534, 414, 288, 166, 560, 434, ...",27


In [201]:
%time group_movie_df3 = group_movie_df3[group_movie_df3['supp'] >= 40].reset_index().drop(columns=['index'])
group_movie_df3

Wall time: 34.5 ms


,movieId,userGroup,supp
0,"(1, 2, 356)","{517, 140, 525, 144, 18, 274, 276, 21, 534, 41...",44
1,"(1, 2, 364)","{517, 140, 525, 144, 18, 19, 274, 21, 534, 276...",42
2,"(1, 2, 480)","{135, 140, 525, 144, 18, 274, 276, 21, 534, 41...",44
3,"(1, 33794, 47)","{525, 141, 17, 18, 274, 534, 414, 288, 166, 56...",45
4,"(1, 33794, 4226)","{522, 525, 141, 17, 18, 274, 279, 414, 288, 16...",47
...,...,...,...
68587,"(3996, 2028, 7153)","{129, 263, 137, 144, 414, 160, 425, 428, 560, ...",41
68588,"(4011, 2028, 7153)","{522, 140, 141, 18, 534, 28, 414, 286, 160, 28...",47
68589,"(59315, 2028, 7153)","{522, 141, 274, 18, 534, 407, 25, 28, 414, 288...",42
68590,"(4022, 2028, 7153)","{517, 263, 140, 141, 274, 28, 414, 286, 288, 2...",43


In [226]:
group_movie_df3.head(10)

,movieId,userGroup,supp
0,"(1, 2, 356)","{517, 140, 525, 144, 18, 274, 276, 21, 534, 41...",44
1,"(1, 2, 364)","{517, 140, 525, 144, 18, 19, 274, 21, 534, 276...",42
2,"(1, 2, 480)","{135, 140, 525, 144, 18, 274, 276, 21, 534, 41...",44
3,"(1, 33794, 47)","{525, 141, 17, 18, 274, 534, 414, 288, 166, 56...",45
4,"(1, 33794, 4226)","{522, 525, 141, 17, 18, 274, 279, 414, 288, 16...",47
5,"(1, 33794, 44191)","{522, 525, 141, 274, 18, 534, 279, 414, 166, 1...",42
6,"(1, 33794, 1196)","{522, 525, 141, 17, 18, 274, 21, 534, 279, 414...",50
7,"(1, 33794, 1198)","{522, 525, 141, 17, 18, 274, 21, 534, 279, 414...",47
8,"(1, 33794, 1210)","{522, 525, 141, 17, 18, 274, 21, 534, 279, 414...",50
9,"(1, 33794, 58559)","{522, 525, 141, 17, 18, 274, 21, 279, 414, 288...",44


In [252]:
# 3 association rule
com = set()
for x in list(group_movie_df3['movieId']):
    com = com.union(set(x))
print(len(com))
print(com)

68592
294
294
{1, 2, 33794, 1028, 6, 3081, 10, 11, 1036, 54286, 16, 21, 32, 34, 39, 3114, 47, 1073, 50, 1079, 1080, 51255, 62, 1089, 1090, 2115, 1097, 3147, 1101, 5218, 1127, 104, 3175, 110, 111, 1136, 49272, 2174, 4226, 2194, 150, 153, 44191, 161, 163, 165, 4262, 8360, 1193, 1196, 1197, 1198, 1199, 1200, 8368, 1201, 1206, 1207, 1208, 185, 1210, 1213, 1214, 58559, 6333, 7361, 1219, 1220, 1221, 1222, 1225, 208, 4306, 1240, 6365, 1246, 223, 1247, 5349, 6373, 231, 6377, 1258, 1259, 235, 6378, 1265, 2291, 1270, 253, 1278, 48385, 5378, 260, 1288, 48394, 1291, 7438, 2324, 2329, 1307, 79132, 288, 292, 293, 72998, 296, 5418, 300, 2355, 73017, 316, 317, 318, 5445, 329, 1356, 3408, 337, 339, 344, 68954, 1370, 2396, 349, 1374, 350, 356, 357, 1387, 364, 367, 368, 1393, 1391, 1394, 377, 380, 48516, 91529, 6539, 3471, 3481, 410, 434, 440, 442, 8636, 2502, 454, 3527, 457, 51662, 8665, 474, 480, 2529, 1517, 2542, 5618, 46578, 500, 1527, 3578, 508, 69122, 2571, 55820, 527, 539, 541, 1573, 5669, 551, 15

# Apiori

In [134]:
import pandas as pd
import numpy as np
from datetime import datetime
from itertools import combinations, permutations

In [76]:
minSupp = 150
minConf = 0.8

In [8]:
k = 1

In [264]:
def countMovies(movieId):
#     global k
#     if k % 10000 == 0:
#         print(k//10000, end=" ")
#     k += 1
    mId = list(movieId)
    mIntersection = movieData['userGroup'][mId[0]]
    for movie in mId[1:]:
        mIntersection = mIntersection.intersection(movieData['userGroup'][movie])
    return mIntersection

In [46]:
allMovies = pd.read_csv("ml-latest-small/movies.csv")
allMovies = allMovies.set_index("movieId")

In [77]:
# read user ratings
allRatingsMovies = pd.read_csv("ml-latest-small/ratings.csv")

start = datetime.now()
# removing all movies that has rating < 3
allRatingsMovies = allRatingsMovies[allRatingsMovies['rating']>=3]

# group movies as per the movieId to combine the user ids into a set to know the users that have rated that movie
groupMovies = allRatingsMovies.groupby('movieId')['userId'].apply(set).reset_index(name="userGroup")
groupMovies['supp'] = groupMovies['userGroup'].apply(lambda x: len(x))
# remove the movieIds where the support is less than minSupp
groupMovies = groupMovies[groupMovies['supp'] >= minSupp].set_index('movieId')

movieData = groupMovies.copy()
time_diff = (datetime.now() - start).total_seconds()
print("Time taken for initial data processing: {} seconds".format(time_diff))

Time taken for initial data processing: 0.480175 seconds


In [267]:
movieCombination = set(list(groupMovies.index))
association_count = 2
association_rules = list()
movieIdSuppPrev = groupMovies['supp'].copy()

while len(movieCombination) > 0:
#     if association_count == 4:
#         break
    print("Calculation for {} itemset!!!".format(association_count))
    # create a dataframe containing the associations as per the association count
    groupMoviesInter = None
    groupMoviesInter = pd.Series(list(combinations(list(movieCombination), association_count))).to_frame("movieId")
    print("Initial number of combinations of itemsets as per association of {}: {}".format(association_count, len(groupMoviesInter)))
    # create set of user groups for each association set and find out the count of each association set as per the no. of users rated
    start = datetime.now()
    groupMoviesInter['userGroup'] = groupMoviesInter['movieId'].apply(lambda x: countMovies(x))
    time_diff = ((datetime.now() - start).total_seconds())/60.0
    print("Time taken to get the usergroup: {} minutes".format(time_diff))
    groupMoviesInter['supp'] = groupMoviesInter['userGroup'].apply(lambda x: len(x))
    groupMoviesInter = groupMoviesInter[groupMoviesInter['supp'] >= minSupp]
    print("Total {} associations >= minSup = {} found: {}".format(association_count, minSupp, len(groupMoviesInter)))
    
    # add the association rules into the association rule list
    if len(groupMoviesInter) == 0:
        break
    groupMoviesInter = groupMoviesInter.reset_index(drop=True).set_index('movieId')
    for movie_id in groupMoviesInter.index:
        idx = tuple(sorted(movie_id))
        for mid in list(combinations(idx, association_count-1)):
            if len(mid) == 1:
                denominator = movieIdSuppPrev.loc[mid]
            else:
                denominator = movieIdSuppPrev.loc[[mid]].iloc[0]['supp']
            conf = groupMoviesInter.loc[[movie_id]].iloc[0]['supp'] * 1.0 / denominator
            if conf >= minConf:
                final = list(mid)
                final.extend(list(set(idx).difference(set(final))))
                association_rules.append(tuple(final))
    print("length of association rules: ", len(association_rules))
    
#     groupMoviesInter['conf'] = groupMoviesInter.apply(lambda x: x['supp'] * 1.0 / movieIdSuppPrev.loc[x['movieId'][:association_count-1]]['supp'], axis=1)
#     groupMoviesInter = groupMoviesInter[groupMoviesInter['conf']>=minConf]
#     print(groupMoviesInter[['movieId', 'supp', 'conf']])
#     association_rules.extend(list(groupMoviesInter.index))
    
    # get the unique movie ids after the association
    movieCombination = set()
    for x in list(groupMoviesInter.index):
        movieCombination = movieCombination.union(set(x))
    print("Number of unique movie ids left after {} association: {}".format(association_count, len(movieCombination)))
    # update the previous dataframe of movie Id and their support
    movieIdSuppPrev = None
    movieIdSuppPrev = groupMoviesInter[['supp']].copy().reset_index(drop=False)
    movieIdSuppPrev['movieId'] = movieIdSuppPrev['movieId'].apply(lambda x: tuple(sorted(x)))
    movieIdSuppPrev = movieIdSuppPrev.set_index('movieId')
    association_count += 1
    print()
print("\nTotal association rules found: {}".format(len(association_rules)))

Calculation for 2 itemset!!!
Initial number of combinations of itemsets as per association of 2: 666
Time taken to get the usergroup: 0.0004410833333333333 minutes
Total 2 associations >= minSup = 150 found: 30
length of association rules:  12
Number of unique movie ids left after 2 association: 19

Calculation for 3 itemset!!!
Initial number of combinations of itemsets as per association of 3: 969
Time taken to get the usergroup: 0.0009413666666666667 minutes
Total 3 associations >= minSup = 150 found: 2
length of association rules:  14
Number of unique movie ids left after 3 association: 4

Calculation for 4 itemset!!!
Initial number of combinations of itemsets as per association of 4: 1
Time taken to get the usergroup: 1.66e-05 minutes
Total 4 associations >= minSup = 150 found: 0

Total association rules found: 14


In [276]:
allMovies = pd.read_csv("ml-latest-small/movies.csv")
allMovies = allMovies.set_index("movieId")

# print the association rules with movie names
for movie_ids in association_rules:
    length = len(movie_ids)
    for idx in range(length):
        print(allMovies.loc[movie_ids[idx]]['title'], end="")
        if idx == len(movie_ids) - 2:
            print(" -> ", end='')
        elif idx != len(movie_ids) - 1:
            print(" , ", end="")
    print()

Lord of the Rings: The Fellowship of the Ring, The (2001) -> Lord of the Rings: The Two Towers, The (2002)
Lord of the Rings: The Two Towers, The (2002) -> Lord of the Rings: The Fellowship of the Ring, The (2001)
Lord of the Rings: The Fellowship of the Ring, The (2001) -> Lord of the Rings: The Return of the King, The (2003)
Lord of the Rings: The Return of the King, The (2003) -> Lord of the Rings: The Fellowship of the Ring, The (2001)
Star Wars: Episode V - The Empire Strikes Back (1980) -> Star Wars: Episode IV - A New Hope (1977)
Star Wars: Episode VI - Return of the Jedi (1983) -> Star Wars: Episode IV - A New Hope (1977)
Apollo 13 (1995) -> Forrest Gump (1994)
Seven (a.k.a. Se7en) (1995) -> Pulp Fiction (1994)
Star Wars: Episode VI - Return of the Jedi (1983) -> Star Wars: Episode V - The Empire Strikes Back (1980)
Lord of the Rings: The Two Towers, The (2002) -> Lord of the Rings: The Return of the King, The (2003)
Lord of the Rings: The Return of the King, The (2003) -> Lord

In [224]:
print(allMovies.loc[480]['title'], "->", allMovies.loc[356]['title'])

Jurassic Park (1993) -> Forrest Gump (1994)


In [262]:
len(association_rules)

14

In [30]:
allMovies = pd.read_csv("ml-latest-small/movies.csv")
allMovies = allMovies.set_index("movieId")
allMovies

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
...,...,...
193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
193585,Flint (2017),Drama


In [1]:
import random
import numpy as np
import pandas as pd
from datetime import datetime
from itertools import combinations

class Apriori:
    def __init__(self):
        self.movie_data = None
        self.dataset_size = None


    def read(self):
        # read all movie name data
        print("#################### DATA READ ####################")
        all_movies = pd.read_csv("ml-latest-small/movies.csv")
        all_movies = all_movies.set_index("movieId")
        print("Movie name and genre data read successfully!!!")
        # ead user ratings data
        all_ratings = pd.read_csv("ml-latest-small/ratings.csv")
        print("Ratings data read successfully!!!")
        self.dataset_size = len(all_ratings['userId'].unique())
        return all_ratings, all_movies


    def sampling(self, all_ratings, alpha=0.6):
        print("#################### DATA SAMPLING ####################")
        unique_user_id = set(all_ratings['userId'])
        removed_user_id = random.sample(unique_user_id, int((1-alpha)*len(unique_user_id)))
        print("Number of unique user ids before sampling: ", len(unique_user_id))
        all_ratings = all_ratings[~all_ratings['userId'].isin(removed_user_id)]
        print("Number of user id to be removed for sampling: ", len(removed_user_id))
        unique_user_id = set(all_ratings['userId'])
        print("Number of unique user ids after sampling: ", len(unique_user_id))
        return all_ratings


    def print_rules(self, rules, all_movies):
        print("#################### ASSOCIATION RULES ####################")
        # print the association rules with movie names
        for i, movie_ids in enumerate(rules):
            length = len(movie_ids)
            print((i+1), end=") ")
            for idx in range(length):
                print(all_movies.loc[movie_ids[idx]]['title'], end="")
                if idx == len(movie_ids) - 2:
                    print(" -> ", end='')
                elif idx != len(movie_ids) - 1:
                    print(" , ", end="")
            print()
            

    def basket_count(self, movie_id):
        m_id = list(movie_id)
        m_intersection = self.movie_data['userGroup'][m_id[0]]
        for movie in m_id[1:]:
            m_intersection = m_intersection.intersection(self.movie_data['userGroup'][movie])
        return len(m_intersection)*1.0/self.dataset_size


    def preprocessing(self, rating_data, min_supp=0.16):
#         print(self.dataset_size)
        print("#################### DATA PREPROCESSING ####################")
        # preprocessing
        start = datetime.now()
        # removing all movies that has rating < 3
        rating_data = rating_data[rating_data['rating']>=3]

        # group movies as per the movieId to combine the user ids into a set to know the users that have rated that movie
        rating_group_data = rating_data.groupby('movieId')['userId'].apply(set).reset_index(name="userGroup")
        rating_group_data['supp'] = rating_group_data['userGroup'].apply(lambda x: len(x)*1.0/self.dataset_size)

        # remove the movieIds where the support is less than minSupp
        rating_group_data = rating_group_data[rating_group_data['supp'] >= min_supp].set_index('movieId')

        time_diff = (datetime.now() - start).total_seconds()
        print("Time taken for initial data processing: {} seconds".format(time_diff))

        return rating_group_data


    def apriori(self, rating_group_data, min_supp=0.16, min_conf=0.95):
        print("#################### APRIORI ####################")
        self.movie_data = rating_group_data.copy()
        movie_combination = set(list(rating_group_data.index))
        assoc_count = 2
        rules = list()
        movie_data_prev = rating_group_data['supp'].copy()

        while len(movie_combination) > 0:
            print("Calculation for {}-itemset!!!".format(assoc_count))
            # create a dataframe containing the associations as per the association count
            group_movies_inter = None
            group_movies_inter = pd.Series(list(combinations(list(movie_combination), assoc_count))).to_frame("movieId")
            print("Initial candidate itemsets of {}-association: {}".format(assoc_count, len(group_movies_inter)))
            # create set of user groups for each association set and find out the count of each association set as per the no. of users rated
            start = datetime.now()
            group_movies_inter['supp'] = group_movies_inter['movieId'].apply(lambda x: self.basket_count(x))
            time_diff = ((datetime.now() - start).total_seconds())/60.0
            print("Time taken to calculate the user basket count: {} minutes".format(time_diff))
            # group_movies_inter['supp'] = group_movies_inter['userGroup'].apply(lambda x: len(x))
            group_movies_inter = group_movies_inter[group_movies_inter['supp'] >= min_supp]
            print("Final Candidate set of {}-associations >= {}(minSup) found: {}".format(assoc_count, min_supp, len(group_movies_inter)))

            # add the association rules into the association rule list
            if len(group_movies_inter) == 0:
                break
            start = datetime.now()
            group_movies_inter = group_movies_inter.reset_index(drop=True).set_index('movieId')
            for movie_id in group_movies_inter.index:
                idx = tuple(sorted(movie_id))
                for mid in list(combinations(idx, assoc_count-1)):
                    if len(mid) == 1:
                        denominator = movie_data_prev.loc[mid]
                    else:
                        denominator = movie_data_prev.loc[[mid]].iloc[0]['supp']
                    conf = group_movies_inter.loc[[movie_id]].iloc[0]['supp'] * 1.0 / denominator
                    if conf >= min_conf:
                        final = list(mid)
                        final.extend(list(set(idx).difference(set(final))))
                        rules.append(tuple(final))
            print("Length of association rules: ", len(rules))
            time_diff = ((datetime.now() - start).total_seconds())/60.0
            print("Time taken to get the association rules with minimum confidence of {}: {} minutes".format(min_conf, time_diff))

            # get the unique movie ids after the association
            movie_combination = set()
            for x in list(group_movies_inter.index):
                movie_combination = movie_combination.union(set(x))
            print("Number of unique movie ids left after {}-association: {}".format(assoc_count, len(movie_combination)))
            # update the previous dataframe of movie Id and their support
            movie_data_prev = None
            movie_data_prev = group_movies_inter[['supp']].copy().reset_index(drop=False)
            movie_data_prev['movieId'] = movie_data_prev['movieId'].apply(lambda x: tuple(sorted(x)))
            movie_data_prev = movie_data_prev.set_index('movieId')
            assoc_count += 1
            print()
        print("\nTotal association rules found: {}\n".format(len(rules)))
        return rules

In [4]:
candidate = Apriori()
all_ratings, all_movies = candidate.read()
rating_group_data = candidate.preprocessing(all_ratings, 0.24)
rules_set = candidate.apriori(rating_group_data, 0.24, 0.8)

#################### DATA READ ####################
Movie name and genre data read successfully!!!
Ratings data read successfully!!!
#################### DATA PREPROCESSING ####################
Time taken for initial data processing: 1.601387 seconds
#################### APRIORI ####################
Calculation for 2-itemset!!!
Initial candidate itemsets of 2-association: 703
Time taken to calculate the user basket count: 0.0014992 minutes
Final Candidate set of 2-associations >= 0.24(minSup) found: 31
Length of association rules:  12
Time taken to get the association rules with minimum confidence of 0.8: 0.0017597333333333332 minutes
Number of unique movie ids left after 2-association: 19

Calculation for 3-itemset!!!
Initial candidate itemsets of 3-association: 969
Time taken to calculate the user basket count: 0.0032372499999999997 minutes
Final Candidate set of 3-associations >= 0.24(minSup) found: 2
Length of association rules:  14
Time taken to get the association rules with mini

In [5]:
candidate.print_rules(rules_set, all_movies)

#################### ASSOCIATION RULES ####################
1) Lord of the Rings: The Fellowship of the Ring, The (2001) -> Lord of the Rings: The Two Towers, The (2002)
2) Lord of the Rings: The Two Towers, The (2002) -> Lord of the Rings: The Fellowship of the Ring, The (2001)
3) Lord of the Rings: The Fellowship of the Ring, The (2001) -> Lord of the Rings: The Return of the King, The (2003)
4) Lord of the Rings: The Return of the King, The (2003) -> Lord of the Rings: The Fellowship of the Ring, The (2001)
5) Star Wars: Episode V - The Empire Strikes Back (1980) -> Star Wars: Episode IV - A New Hope (1977)
6) Star Wars: Episode VI - Return of the Jedi (1983) -> Star Wars: Episode IV - A New Hope (1977)
7) Apollo 13 (1995) -> Forrest Gump (1994)
8) Seven (a.k.a. Se7en) (1995) -> Pulp Fiction (1994)
9) Star Wars: Episode VI - Return of the Jedi (1983) -> Star Wars: Episode V - The Empire Strikes Back (1980)
10) Lord of the Rings: The Two Towers, The (2002) -> Lord of the Rings: The R

In [47]:
import random

all_ratings, all_movies = candidate.read()

keep_rate = 0.9
unique_user_id = set(all_ratings['userId'])
removed_user_id = random.sample(unique_user_id, int((1-keep_rate)*len(unique_user_id)))
print("Length of user ids before sampling: ", len(unique_user_id))
all_ratings = all_ratings[~all_ratings['userId'].isin(removed_user_id)]
print("Length of user id to be removed: ", len(removed_user_id))
unique_user_id = set(all_ratings['userId'])
print("Length of user ids after sampling: ", len(unique_user_id))

Length of user ids before sampling:  610
Length of user id to be removed:  60
Length of user ids after sampling:  550


In [48]:
all_ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [49]:
{1,2,3}.difference({2,3})

{1}